In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate, FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler, get_openai_callback
from langchain.prompts import PromptTemplate,ChatPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector
from langchain.prompts import load_prompt
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm


chat = OpenAI(
    temperature=0.1, 
    max_tokens=450,
    model="gpt-3.5-turbo"
)

chat.save("model.json")


Soju is a traditional Korean distilled alcoholic beverage. The recipe for soju typically involves the following ingredients and steps:

Ingredients:
- 1 kg of rice (short-grain rice is commonly used)
- 1 kg of nuruk (a Korean fermentation starter)
- 10 liters of water
- 400-500 grams of sugar (optional, for sweetness)
- Distilled water (for dilution)

Steps:
1. Rinse the rice thoroughly to remove any impurities.
2. Cook the rice in a rice cooker or on the stovetop until it is fully cooked and slightly sticky.
3. Allow the cooked rice to cool down to room temperature.
4. In a large container or pot, mix the cooled rice with nuruk. Make sure to break up any clumps of nuruk and mix it well with the rice.
5. Add water to the rice and nuruk mixture, stirring it gently to combine everything.
6. Cover the container or pot with a clean cloth or plastic wrap and let it ferment for about 7-10 days at room temperature. During this time, the mixture will undergo fermentation.
7. After the fermenta

In [27]:
chat.predict("How do you make italian pasta?")

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n\n2. Crack the eggs into the well and add the salt.\n\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little at a time until a dough starts to form.\n\n4. Once the dough starts to come together, knead it with your hands for about 5-10 minutes until it becomes smooth and elastic. If the dough feels too dry, you can add a little water, teaspoon by teaspoon, until it reaches the desired consistency. If it's too sticky, add a little more flour.\n\n5. Once the dough is ready, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes to an hour. This allows the gluten to relax and makes the dough easie